#### Exercise 8.1

To get used to thinking about testing in this way, come up with properties that specify the implementation of a `sum: List[Int] => Int` function. You don’t have to write your properties down as executable ScalaCheck code—an informal description is fine. Here are some ideas to get you started:

* Reversing a list and summing it should give the same result as summing the original, non-reversed list.
* What should the sum be if all elements of the list are the same value?
* Can you think of other properties?

##### Answer

1. Sum of an empty list is zero
2. Sum of a list should equal to its head plus the sum of the tail
3. Sum of repeated number should be `n * number`

#### Exercise 8.2

What properties specify a function that finds the maximum of a `List[Int]`?

##### Answer

1. Maximum is greater or equal to all items in a list
2. Maximum number is an element in the list
3. Maximum of empty list is undefined

#### Exercise 8.3

Assuming the following representation of `Prop`, implement `&&` as a method of `Prop`.

In [1]:
trait Prop:
    self =>
    def check: Boolean
    def &&(that: Prop): Prop = 
        new Prop:
            def check = self.check && that.check

defined trait Prop

In [2]:
object Prop:
    type FailedCase = String
    type SuccessCount = Int

trait Prop:
    self =>

    import Prop.*
    def check: Either[(FailedCase, SuccessCount), SuccessCount]
    def &&(that: Prop): Prop

defined object Prop
defined trait Prop

In [3]:
type State[S, +A] = S => (A, S)

object State:
    def apply[S, A](f: S => (A, S)): State[S, A] = f

    def unit[S, A](a: A): State[S, A] =
        s => (a, s)

    def sequence[S, A](s: List[State[S, A]]): State[S, List[A]] =
        s.foldRight(unit(Nil : List[A]))((s, acc) => s.map2(acc)(_ :: _))

extension [S, A](underlying: State[S, A])
    def run(s: S): (A, S) = underlying(s)

    def flatMap[B](f: A => State[S, B]): State[S, B] =
        s => 
            val (a, s2) = underlying(s)
            f(a)(s2)
    
    def map[B](f: A => B): State[S, B] =
        underlying.flatMap(a => State.unit(f(a)))

    def map2[B, C](sb: State[S, B])(f: (A, B) => C): State[S, C] =
        underlying.flatMap(a => sb.map(b => f(a, b)))

defined type State
defined object State
defined extension methods 

In [4]:
trait RNG:
    def nextInt: (Int, RNG)

case class SimpleRNG(seed: Long) extends RNG:
    def nextInt: (Int, RNG) =
        val newSeed = (seed * 0x5DeeCE66DL + 0xBL) & 0xFFFFFFFFFFFFL
        val nextRNG = SimpleRNG(newSeed)
        val n = (newSeed >>> 16).toInt
        (n, nextRNG)

def nonNegativeInt(rng: RNG): (Int, RNG) =
    val (n, nextRNG) = rng.nextInt
    val absN = if n == Int.MinValue then 0 else scala.math.abs(n)
    (n, nextRNG)

type Gen[+A] = State[RNG, A]

defined trait RNG
defined class SimpleRNG
defined function nonNegativeInt
defined type Gen

#### Exercise 8.4

Implement `Gen.choose` using this representation of `Gen`. It should generate integers in the range `start` to `stopExclusive`. Feel free to use functions you’ve already written.

In [5]:
def choose(start: Int, stopExclusive: Int): Gen[Int] =
    State(nonNegativeInt).map(_ % (stopExclusive - start) + start)

defined function choose

#### Exercise 8.5

Let’s see what else we can implement using this representation of `Gen`. Try implementing `unit`, `boolean`, and `listOfN`.

```scala
def unit[A](a: => A): Gen[A]                                       
def boolean: Gen[Boolean]
extension [A](self: Gen[A]) def listOfN[A](n: Int): Gen[List[A]]
```

In [6]:
def unit[A](a: => A): Gen[A] = State.unit(a)

defined function unit

In [7]:
def boolean: Gen[Boolean] = choose(0, 2).map(_ == 1)

defined function boolean

In [8]:
extension [A](self: Gen[A]) 
    def listOfN(n: Int): Gen[List[A]] =
        State.sequence(List.fill(n)(self))

defined extension methods 

#### Exercise 8.6

Implement `flatMap`, and then use it to implement this more dynamic version of `listOfN`. Put `flatMap` and `listOfN` in the `Gen` class.

In [9]:
extension [A](self: Gen[A])
    def flatMap[B](f: A => Gen[B]): Gen[B] =
        rng =>
            val (a, r2) = self(rng)
            f(a)(r2)

    def listOfN(size: Gen[Int]): Gen[List[A]] =
        size.flatMap(n => State.sequence(List.fill(n)(self)))

defined extension methods 

#### Exercise 8.7

Implement `union`, for combining two generators of the same type into one, by pulling values from each generator with equal likelihood.
    

In [10]:
def union[A](g1: Gen[A], g2: Gen[A]): Gen[A] =
    boolean.flatMap(if _ then g1 else g2)

defined function union

#### Exercise 8.8

Implement `weighted`, a version of `union` that accepts a weight for each `Gen` and generates values from each `Gen` with probability proportional to its weight.

In [11]:
def weighted[A](g1: (Gen[A], Double), g2: (Gen[A], Double)): Gen[A] =
    val g1Threshold = (Int.MaxValue * g1._2 / (g1._2 + g2._2)).toInt
    choose(0, Int.MaxValue).flatMap(n => if n <= g1Threshold then g1._1 else g2._1)

defined function weighted

In [12]:
enum Result:
    case Passed
    case Falsified(failure: Prop.FailedCase, successes: Prop.SuccessCount)

    def isFalsified: Boolean = this match
        case Passed => false
        case _ => false

type TestCases = Int
object TestCases:
    extension (x: TestCases) def toInt: Int = x
    def fromInt(x: Int): TestCases = x

type Prop = (TestCases, RNG) => Result

defined class Result
defined type TestCases
defined object TestCases
defined type Prop

In [13]:
def forAll[A](as: Gen[A])(f: A => Boolean): Prop =
    import Result.*
    
    (n, rng) =>
        randomLazyList(as)(rng)
            .zip(LazyList.from(0))
            .take(n)
            .map{
                case (a, i) =>
                    try
                        if f(a) then Passed
                        else Falsified(a.toString, i)
                    catch
                        case e: Exception =>
                            Falsified(buildMsg(a, e), i)
            }
            .find(_.isFalsified)
            .getOrElse(Passed)

def randomLazyList[A](g: Gen[A])(rng: RNG): LazyList[A] =
    LazyList.unfold(rng)(rng => Some(g.run(rng)))

def buildMsg[A](s: A, e: Exception): String =
    s"test case: $s\n" +
    s"generated an exception: ${e.getMessage}\n" +
    s"stack trace:\n ${e.getStackTrace.mkString("\n")}"

defined function forAll
defined function randomLazyList
defined function buildMsg

#### Exercise 8.9

Now that we have a representation of `Prop`, implement `&&` and `||` for composing `Prop` values. Notice that in the case of failure we don’t know which property was responsible, the left or the right. Can you devise a way of handling this, perhaps by allowing Prop values to be assigned a tag or label which gets displayed in the event of a failure?

In [14]:
extension (self: Prop)
    def &&(that: Prop): Prop =
        (n, rng) => self(n, rng) match
            case Result.Passed => that(n, rng)
            case f => f

    def ||(that: Prop): Prop =
        (n, rng) => self(n, rng) match
            case Result.Passed => Result.Passed
            case _ => that(n, rng)


defined extension methods 

In [15]:
extension (self: Prop)    
    def run(): Unit =
        self(100, SimpleRNG(System.currentTimeMillis)) match
            case Result.Falsified(msg, n) =>
                println(s"! Falsified after $n passed tests:\n $msg")
            case Result.Passed =>
                println(s"+ OK, passed 100 tests.")

defined extension methods 

In [16]:
val p: Prop = forAll(boolean)(x => x == x)

p.run()

+ OK, passed 100 tests.


p: (TestCases, RNG) => Result = ammonite.$sess.cmd12$Helper$$Lambda$3037/545333784@162d1e54